In [1]:

'''
#首先我们从图片里以字符串的形式来读出图片内容：
import tensorflow as tf
sess = tf.Session()
image_string = tf.read_file("HWDB1/train/00000/702.png")
print(sess.run(image_string))
'''

'\n#首先我们从图片里以字符串的形式来读出图片内容：\nimport tensorflow as tf\nsess = tf.Session()\nimage_string = tf.read_file("HWDB1/train/00000/702.png")\nprint(sess.run(image_string))\n'

In [2]:
'''
#解码
with sess.as_default():
    image_decode = tf.image.decode_image(image_string,channels=1)
    image_resize = tf.image.resize_images(image_decode.eval(),[100,80])
    print(image_resize)
    '''

'\n#解码\nwith sess.as_default():\n    image_decode = tf.image.decode_image(image_string,channels=1)\n    image_resize = tf.image.resize_images(image_decode.eval(),[100,80])\n    print(image_resize)\n    '

In [3]:
'''
#label读取
import pickle
f = open('HWDB1/char_dict','br')
dict = pickle.load(f)
print(dict)
'''

"\n#label读取\nimport pickle\nf = open('HWDB1/char_dict','br')\ndict = pickle.load(f)\nprint(dict)\n"

In [4]:
import tensorflow as tf
import os
def read_data(dataPath):    
    image_files=[]
    label_arr=[]
    for char in os.listdir(dataPath):
        for image in os.listdir(os.path.join(dataPath,char)):
            image_files.append(os.path.join(dataPath,char,image))
            label_arr.append(int(char))
    #images = tf.convert_to_tensor(image_files)
    #labels = tf.convert_to_tensor(label_arr)
    return image_files,label_arr
images,labels = read_data('HWDB1/train')
images_eval,labels_eval = read_data('HWDB1/test')

def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_png(image_string,channels=1)
    image_resized = tf.image.resize_images(image_decoded, [80, 80])
    return image_resized, label

def train_input_fn(images,labels):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    dataset = dataset.shuffle(buffer_size=895035).repeat()
    dataset = dataset.map(_parse_function).batch(100)
    return dataset.make_one_shot_iterator().get_next()

In [5]:
#神经网络
def cnn_model_fn(features, labels, mode):
    input_layer =tf.reshape(features,[-1,80,80,1])
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu,
      name="conv1")
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2,name="pool1")
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu,
      name="conv2")
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],strides=2,name="pool2")
    conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=256,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    pool3_flat = tf.reshape(pool3, [-1, 10 * 10 * 256])
    dense = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu)

    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=3755,name="logits")
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.002)
        train_op = optimizer.minimize(
          loss=loss,
          global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
#模型训练
chart_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="char_3_Conv_2")
chart_classifier.train(input_fn=lambda:train_input_fn(images,labels),steps=10000)    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'char_3_Conv_2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F55FF24EF0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use ker

In [ ]:
batchsize=100
#map  = tf.map_fn(images_eval,labels_eval)
eval_results=chart_classifier.evaluate(input_fn=lambda:train_input_fn(images_eval,labels_eval))
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-03T06:22:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from char_3_Conv_2\model.ckpt-2
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
